In [1]:
%%time

import datetime as dt
import os

import pandas
import pandas as pd
import praw
from praw.models import ListingGenerator
from tqdm import tqdm

from common.captioning.caption import BlipCaption
from common.schemas.pyarrow_schema import schema
from common.storage.azure_file_storage import AzureFileStorageAdapter
from common.functions.functions import Functions

tqdm.pandas(desc="Progress")

file_system = AzureFileStorageAdapter('data').get_file_storage()

functions: Functions = Functions()

CPU times: total: 9.53 s
Wall time: 18.1 s


In [2]:
%%time

# "memes","CityPorn", "EarthPorn", "spaceporn",	"itookapicture", "trippinthroughtime"
subs = [
	"SFWRedheads",
	"sfwpetite",
	"SFWNextDoorGirls",
	"fatsquirrelhate",
	"realasians",
	"KoreanHotties",
	"prettyasiangirls",
	"AsianOfficeLady",
	"mildlypenis",
	"AsianInvasion",
	"sexygirls",
	"PrettyGirls",
	"gentlemanboners",
	"hotofficegirls",
	"tightdresses",
	"DLAH",
	"bathandbodyworks",
	"AesPleasingAsianGirls"
]

CPU times: total: 0 ns
Wall time: 0 ns


In [3]:
%%time

sources = [
	{"name": "CityDiffusion", "data": ["CityPorn"]},
	{"name": "NatureDiffusion", "data": ["EarthPorn"]},
	{"name": "CosmicDiffusion", "data": ["spaceporn"]},
	{"name": "ITAPDiffusion", "data": ["itookapicture"]},
	{"name": "MemeDiffusion", "data": ["memes"]},
	{"name": "TTTDiffusion", "data": ["trippinthroughtime"]},
	{"name": "WallStreetDiffusion", "data": ["wallstreetbets"]},
	{"name": "SexyDiffusion", "data": ["selfies", "Amicute", "amihot", "AmIhotAF", "HotGirlNextDoor"]},
	{"name": "FatSquirrelDiffusion", "data": ["fatsquirrelhate"]},
	{"name": "CelebrityDiffusion", "data": ["celebrities"]},
	{"name": "OldLadyDiffusion", "data": ["oldladiesbakingpies"]},
	{"name": "SWFPetite", "data": ["sfwpetite"]},
	{"name": "SFWMilfs", "data": ["cougars_and_milfs_sfw"]},
	{"name": "RedHeadDiffusion", "data": ["SFWRedheads"]},
	{"name": "NextDoorGirlsDiffusion", "data": ["SFWNextDoorGirls"]},
	{"name": "SexyAsianDiffusion","data": ["realasians", "KoreanHotties", "prettyasiangirls", "AsianOfficeLady", "AsianInvasion","AesPleasingAsianGirls"]},
	{"name": "MildlyPenisDiffusion", "data": ["mildlypenis"]},
	{"name": "PrettyGirlDiffusion","data": ["sexygirls", "PrettyGirls", "gentlemanboners", "hotofficegirls", "tightdresses", "DLAH"]},
	{"name": "CandleDiffusion", "data": ["bathandbodyworks"]}
]
sources_df = pd.DataFrame.from_records(sources)

CPU times: total: 0 ns
Wall time: 10 ms


In [4]:
%%time

extant_data = pandas.read_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system,
								  schema=schema)
extant_data.set_index('id', inplace=True)
display(extant_data)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]


CPU times: total: 688 ms
Wall time: 1.4 s


In [5]:
%%time

curated_data = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system, schema=schema)
curated_data.set_index('id', inplace=True)

display("==== CURATED DATA ====")
display(curated_data.shape)
display(curated_data)

'==== CURATED DATA ===='

(39632, 14)

,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,
100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13n9x6y,bathandbodyworks,PhysicalCharacter3,Anyone buy or try this one? 👀,a close up of a jar of candles on a store shelf,77b37fb1987d8d0d8aff7cadd6942b95,/r/bathandbodyworks/comments/13n9x6y/anyone_bu...,https://i.redd.it/paiuac5fk31b1.jpg,13n9x6y.jpg,data/image/13n9x6y.jpg,CandleDiffusion,True,False,False,[]
13nc2ad,bathandbodyworks,TFCosplayer92,Just a SMALL Firecracker Pop Haul 🤣,arafed items are laid out on the floor in a room,e4e276e4dbd240d546248f2951212e2e,/r/bathandbodyworks/comments/13nc2ad/just_a_sm...,https://i.redd.it/lf40r0xk041b1.jpg,13nc2ad.jpg,data/image/13nc2ad.jpg,CandleDiffusion,True,False,False,[]
13nczq7,bathandbodyworks,blondenextdoor30,Happy Saturday from the West Coast!🌞🩶🍾 What’s ...,candles are lit on a counter top with a gold lid,8bcf2db56ca1390ad579fa44a0d5716d,/r/bathandbodyworks/comments/13nczq7/happy_sat...,https://i.redd.it/7b78c8l2841b1.jpg,13nczq7.jpg,data/image/13nczq7.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 594 ms
Wall time: 977 ms


In [6]:
%%time

extant_data.update(curated_data)
updated_extant_data = extant_data.reset_index()

display("==== UPDATED EXTANT DATA ====")
display(updated_extant_data.shape)
display(updated_extant_data)
updated_extant_data.to_parquet("data/processed_raw_data.parquet", engine='pyarrow', filesystem=file_system, schema=schema)

'==== UPDATED EXTANT DATA ===='

(41505, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41500,131dso7,DLAH,Wallydinger123,Happy B Day,there are three women posing for a picture in ...,ebb4b4a88dc79e2f2f57b10f32bf03c0,/r/DLAH/comments/131dso7/happy_b_day/,https://i.redd.it/36gurz533jwa1.jpg,131dso7.jpg,data/image/131dso7.jpg,PrettyGirlDiffusion,True,True,False,[]
41501,131eznp,DLAH,Long_dong111,Perfect,araffe woman in a white dress sitting on a pur...,5c944c6d62b1ae182c5a6a40c64195f4,/r/DLAH/comments/131eznp/perfect/,https://i.redd.it/dnbbx4zucjwa1.jpg,131eznp.jpg,data/image/131eznp.jpg,PrettyGirlDiffusion,True,True,True,[]
41502,131g3am,DLAH,ThrownAwayMiles,DLAH left or right,two women in short dresses posing for a pictur...,ab1f8fc33c85f4cfce12c810cf22229e,/r/DLAH/comments/131g3am/dlah_left_or_right/,https://i.redd.it/zpp70oor3lwa1.jpg,131g3am.jpg,data/image/131g3am.jpg,PrettyGirlDiffusion,True,True,True,[]
41503,131jvcx,DLAH,robok212,Good morning 😍,araffe in a pink dress posing on a white wall,01f5058855526ef25888cff88065db4c,/r/DLAH/comments/131jvcx/good_morning/,https://i.redd.it/ozmbfhjv3mwa1.jpg,131jvcx.jpg,data/image/131jvcx.jpg,PrettyGirlDiffusion,True,True,False,[]


CPU times: total: 781 ms
Wall time: 3.39 s


In [7]:
%%time

reddit: praw.Reddit = praw.Reddit(client_id='5hVavL0PIRyM_1JSvqT6UQ', client_secret='BjD2kS3WNLnJc59RKY-JJUuc_Z9-JA', user_agent='script:%(bot_name)s:v%(bot_version)s (by /u/%(bot_author)s)')

CPU times: total: 62.5 ms
Wall time: 133 ms


In [8]:
for sub in tqdm(subs, total=len(subs), desc="Creating Temp Dir For Subs..."):
	temp_dir_path_ = f"temp/{sub}"
	if not os.path.exists(temp_dir_path_):
		os.makedirs(temp_dir_path_)
	os.makedirs(temp_dir_path_, exist_ok=True)

Creating Temp Dir For Subs...: 100%|██████████| 18/18 [00:00<00:00, 3600.95it/s]


In [9]:
with open('log.txt', 'a') as f:
	for time_filter in ['week', 'day']:
		for sub in tqdm(subs, desc=f"{time_filter}", total=len(subs)):
			temp_dir_path = f"temp/{sub}"
			try:
				subreddit_stream: ListingGenerator = reddit.subreddit(display_name=sub).top(limit=100, time_filter=time_filter)
				subreddit_stream = list(subreddit_stream)
				for submission in tqdm(subreddit_stream, total=len(subreddit_stream), desc=f"Posts - {sub} - {time_filter}"):
					if submission is None:
						continue
					else:
						if submission.id in extant_data.index.values:
							continue
					try:
						author_name = 'Unknown'
						subreddit_name = sub
						try:
							author_name = submission.author.name
						except Exception as e:
							author_name = 'Unknown'
							pass
						p = {
							'id': submission.id,
							'subreddit': subreddit_name,
							'author': author_name,
							'title': submission.title,
							'caption': '',
							'hash': '',
							'permalink': submission.permalink,
							'original_url': submission.url,
							'image_name': '',
							'path': '',
							'thumbnail_path': '',
							'exists': False,
							'curated': False,
							'Tags': ['']
						}
						pd.DataFrame([p]).to_parquet(f"{temp_dir_path}/{submission.id}.parquet")
					except Exception as e:
						log = functions.write_log_message(submission.id, sub, "Error Writing Post", e)
						f.write(log)
						continue
			except Exception as e:
				log = functions.write_log_message(submission.id, sub, f"Error Getting Posts For SubReddit", e)
				f.write(log)
				continue

day: 100%|██████████| 18/18 [03:09<00:00, 10.50s/it]


In [10]:
%%time

data = []
for sub in tqdm(subs, desc="Reading Temp Dir For Subs..."):
	df = pandas.read_parquet(f"temp/{sub}", schema=schema, engine='pyarrow')
	records = df.to_dict(orient='records')
	data.extend(records)

temp_data = pandas.DataFrame(data)

for i, r in temp_data.iterrows():
	temp_data.loc[i, 'image_name'] = r.id + ".jpg"
	temp_data.loc[i, 'path'] = ""
	temp_data.loc[i, 'hash'] = ""
	temp_data.loc[i, 'caption'] = ""
	temp_data.loc[i, 'model'] = ""
	temp_data.loc[i, 'exists'] = False
	temp_data.loc[i, 'curated'] = False
	temp_data.loc[i, 'accept'] = False
	temp_data.loc[i, 'tags'] = ['']

temp_data.set_index('id', inplace=True, drop=False)

display("==== TEMP DATA ====")
display(temp_data.shape)
display(temp_data)

Reading Temp Dir For Subs...: 100%|██████████| 18/18 [00:02<00:00,  8.14it/s]


'==== TEMP DATA ===='

(1895, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13h67sb,13h67sb,SFWRedheads,Dasharkiss_real,Love life and life will love you too.,,,/r/SFWRedheads/comments/13h67sb/love_life_and_...,https://i.imgur.com/NBCDz3f.jpg,13h67sb.jpg,,,False,False,False,[]
13h93zj,13h93zj,SFWRedheads,thatgirlmarie97,Who wants to play some games?,,,/r/SFWRedheads/comments/13h93zj/who_wants_to_p...,https://www.reddit.com/gallery/13h93zj,13h93zj.jpg,,,False,False,False,[]
13h9lq5,13h9lq5,SFWRedheads,snildeep,Hey! My eyes are up here! 😂💕,,,/r/SFWRedheads/comments/13h9lq5/hey_my_eyes_ar...,https://i.redd.it/schip1qaotza1.jpg,13h9lq5.jpg,,,False,False,False,[]
13hc2uz,13hc2uz,SFWRedheads,nurselacey,How do you like my haircut?,,,/r/SFWRedheads/comments/13hc2uz/how_do_you_lik...,https://i.redd.it/0qgvz4uy9uza1.jpg,13hc2uz.jpg,,,False,False,False,[]
13hcvxr,13hcvxr,SFWRedheads,Puzzled_MJ,Happy Sunday. Where would u take me today?,,,/r/SFWRedheads/comments/13hcvxr/happy_sunday_w...,https://i.redd.it/h6bwkyw8guza1.jpg,13hcvxr.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13mh4fw,13mh4fw,AesPleasingAsianGirls,Korean_Celebrity_Fap,Arin Compilation,,,/r/AesPleasingAsianGirls/comments/13mh4fw/arin...,https://i.imgur.com/bukbo8v.gifv,13mh4fw.jpg,,,False,False,False,[]
13n9xdx,13n9xdx,AesPleasingAsianGirls,yummybanchan,Song Yu Qi,,,/r/AesPleasingAsianGirls/comments/13n9xdx/song...,https://i.redd.it/xl0v8fgbk31b1.jpg,13n9xdx.jpg,,,False,False,False,[]
13nm990,13nm990,AesPleasingAsianGirls,Defiant-Appearance64,Somi compilation,,,/r/AesPleasingAsianGirls/comments/13nm990/somi...,https://i.imgur.com/ISJoDGW.gifv,13nm990.jpg,,,False,False,False,[]


CPU times: total: 15.2 s
Wall time: 8.31 s


In [11]:
%%time

filtered = temp_data[~temp_data['id'].isin(extant_data.index.values) & temp_data['original_url'].str.endswith('.jpg') & (~temp_data['id'].isin(curated_data.index.values))]
filtered.dropna(axis=1, how='all')
filtered.reset_index(drop=True, inplace=True)
filtered.set_index('id', inplace=True, drop=False)
display("==== FILTERED DATA ====")
display(filtered.shape)
display(filtered)

'==== FILTERED DATA ===='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,,,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,,,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",,,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,,,False,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,,,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,,,False,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,,,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,,,False,False,False,[]


CPU times: total: 78.1 ms
Wall time: 94 ms


In [12]:
%%time

filtered['model'] = filtered.progress_apply(lambda x: functions.add_source(x, sources), axis=1)

display("== Filtered Data With Model ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 99/99 [00:00<00:00, 16502.51it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Model =='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,,,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",,,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,,PrettyGirlDiffusion,False,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,,,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,,CandleDiffusion,False,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,,,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,,CandleDiffusion,False,False,False,[]


CPU times: total: 78.1 ms
Wall time: 75 ms


In [13]:
%%time

file_list = file_system.ls("data/image")

filtered['path'] = filtered.progress_apply(lambda x: functions.fetch_image(x, file_list, file_system), axis=1)

display("== Filtered Data With Path ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 99/99 [00:51<00:00,  1.94it/s]
<timed exec>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Path =='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,,,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,False,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",,,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,False,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,,,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,False,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,,,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,False,False,False,[]


CPU times: total: 1min 2s
Wall time: 1min 54s


In [14]:
%%time

filtered['exists'] = filtered.progress_apply(lambda x: functions.set_exists(x), axis=1)

display("== Filtered Data With Exists ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 99/99 [00:00<00:00, 9897.89it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Exists =='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,,,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",,,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,,,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,,,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 62.5 ms
Wall time: 53 ms


In [15]:
%%time

filtered['hash'] = filtered.progress_apply(lambda x: functions.set_hash(x), axis=1)

display("== Filtered Data With Hash ==")
display(filtered.shape)
display(filtered)

Progress: 100%|██████████| 99/99 [00:00<00:00, 565.73it/s]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Hash =='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,,955d683562bc7177e5369c129030c888,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 219 ms
Wall time: 209 ms


In [16]:
%%time
import torch
if torch.cuda.is_available():
	caption_0 = BlipCaption("cuda:0")
	caption_1 = BlipCaption("cuda:1")
else:
	caption_0 = BlipCaption("cpu")
	caption_1 = BlipCaption("cpu")

CPU times: total: 1min 33s
Wall time: 1min 32s


In [17]:
%%time

filtered['caption'] = filtered.progress_apply(lambda x: functions.apply_caption(x, [caption_0, caption_1]), axis=1)

display("== Filtered Data With Caption ==")
display(filtered.shape)
display(filtered)

Progress:   0%|          | 0/99 [00:00<?, ?it/s]D:\code\repos\simple-collection\venv\lib\site-packages\transformers\generation\utils.py:1313: UserWarning: Using `max_length`'s default (20) to control the generation length. This behaviour is deprecated and will be removed from the config in v5 of Transformers -- we recommend using `max_new_tokens` to control the maximum length of the generation.
  warnings.warn(
Progress: 100%|██████████| 99/99 [02:12<00:00,  1.34s/it]
<timed exec>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


'== Filtered Data With Caption =='

(99, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13hgme0,13hgme0,SFWRedheads,Redhotwife88,I Love wearing my Leather Pants and Heels! [F],,,/r/SFWRedheads/comments/13hgme0/i_love_wearing...,https://i.redgifs.com/i/frayedmadeupsparrow.jpg,13hgme0.jpg,,RedHeadDiffusion,False,False,False,[]
13ibnsi,13ibnsi,SFWRedheads,Dasharkiss_real,I can live without money but I can't live with...,,,/r/SFWRedheads/comments/13ibnsi/i_can_live_wit...,https://i.redgifs.com/i/imaginaryrawleveret.jpg,13ibnsi.jpg,,RedHeadDiffusion,False,False,False,[]
13imeet,13imeet,SFWRedheads,imjenrose,Adorable girl to make your day better,,,/r/SFWRedheads/comments/13imeet/adorable_girl_...,https://i.redgifs.com/i/dangerousfilthyhornedv...,13imeet.jpg,,RedHeadDiffusion,False,False,False,[]
13jm7mh,13jm7mh,SFWRedheads,Redhotwife88,Do I look Cute for my Anniversary Dinner? [F],,,/r/SFWRedheads/comments/13jm7mh/do_i_look_cute...,https://i.redgifs.com/i/tensedarkmagentabedlin...,13jm7mh.jpg,,RedHeadDiffusion,False,False,False,[]
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,arafed woman sitting in a car with her legs cr...,955d683562bc7177e5369c129030c888,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 3min 11s
Wall time: 2min 12s


In [18]:
%%time

filtered_more = filtered.loc[(filtered['caption'] != "") & (filtered['caption'].notnull()) & (filtered['exists'] == True)]

display("== Filtered On Missing Caption ==")
display(filtered_more.shape)
display(filtered_more)

'== Filtered On Missing Caption =='

(87, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,arafed woman sitting in a car with her legs cr...,955d683562bc7177e5369c129030c888,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
13nifj6,13nifj6,SFWRedheads,arayofsexysunshine,What a crazy and touching time I’ve had on the...,arafed woman with red hair wearing a green jac...,f59f2c71f8c64da25432c72fa2ddd099,/r/SFWRedheads/comments/13nifj6/what_a_crazy_a...,https://i.redd.it/f2anwulzh51b1.jpg,13nifj6.jpg,data/image/13nifj6.jpg,RedHeadDiffusion,True,False,False,[]
13njz51,13njz51,SFWRedheads,madelynlouise,Went to the grocery store in this,there is a woman taking a selfie in a mirror,7e70167383380384e1f6fe47455faf0f,/r/SFWRedheads/comments/13njz51/went_to_the_gr...,https://i.redd.it/wcr5fnxcw51b1.jpg,13njz51.jpg,data/image/13njz51.jpg,RedHeadDiffusion,True,False,False,[]
13nka7r,13nka7r,SFWRedheads,celluxxx,Felt cute in this outfit ❤️,there is a woman sitting on a bed with a red d...,49bed56ba79c6c236ca818ed33b2cd04,/r/SFWRedheads/comments/13nka7r/felt_cute_in_t...,https://i.redd.it/m1lefhhaz51b1.jpg,13nka7r.jpg,data/image/13nka7r.jpg,RedHeadDiffusion,True,False,False,[]
13np8z0,13np8z0,SFWRedheads,Junior-Midnight-8241,Do I look innocent? 👀,arafed woman with long red hair and white top ...,17c5c634443b786c265dbe4bded9f20f,/r/SFWRedheads/comments/13np8z0/do_i_look_inno...,https://i.redd.it/nf7kb3tub71b1.jpg,13np8z0.jpg,data/image/13np8z0.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 62.5 ms
Wall time: 45 ms


In [19]:
%%time

dropped = filtered_more.dropna(axis=1, how='all')
dropped.reset_index(drop=True, inplace=True)

display("== Dropped And Reset Data ==")
display(dropped.shape)
display(dropped)

'== Dropped And Reset Data =='

(87, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,arafed woman sitting in a car with her legs cr...,955d683562bc7177e5369c129030c888,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
1,13nifj6,SFWRedheads,arayofsexysunshine,What a crazy and touching time I’ve had on the...,arafed woman with red hair wearing a green jac...,f59f2c71f8c64da25432c72fa2ddd099,/r/SFWRedheads/comments/13nifj6/what_a_crazy_a...,https://i.redd.it/f2anwulzh51b1.jpg,13nifj6.jpg,data/image/13nifj6.jpg,RedHeadDiffusion,True,False,False,[]
2,13njz51,SFWRedheads,madelynlouise,Went to the grocery store in this,there is a woman taking a selfie in a mirror,7e70167383380384e1f6fe47455faf0f,/r/SFWRedheads/comments/13njz51/went_to_the_gr...,https://i.redd.it/wcr5fnxcw51b1.jpg,13njz51.jpg,data/image/13njz51.jpg,RedHeadDiffusion,True,False,False,[]
3,13nka7r,SFWRedheads,celluxxx,Felt cute in this outfit ❤️,there is a woman sitting on a bed with a red d...,49bed56ba79c6c236ca818ed33b2cd04,/r/SFWRedheads/comments/13nka7r/felt_cute_in_t...,https://i.redd.it/m1lefhhaz51b1.jpg,13nka7r.jpg,data/image/13nka7r.jpg,RedHeadDiffusion,True,False,False,[]
4,13np8z0,SFWRedheads,Junior-Midnight-8241,Do I look innocent? 👀,arafed woman with long red hair and white top ...,17c5c634443b786c265dbe4bded9f20f,/r/SFWRedheads/comments/13np8z0/do_i_look_inno...,https://i.redd.it/nf7kb3tub71b1.jpg,13np8z0.jpg,data/image/13np8z0.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
83,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
84,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]
85,13npfx3,bathandbodyworks,singshopsleep,All of Today's Offers,a poster with a variety of products for sale o...,635af0f6bb1a2b81c9ce46bd29374d72,/r/bathandbodyworks/comments/13npfx3/all_of_to...,https://i.redd.it/skqywk3qd71b1.jpg,13npfx3.jpg,data/image/13npfx3.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 31.2 ms
Wall time: 34 ms


In [20]:
%%time

current = pandas.read_parquet("data/parquet/back.parquet", engine='pyarrow', filesystem=file_system)

display(current.shape)
display(current)

(39632, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39627,13n9x6y,bathandbodyworks,PhysicalCharacter3,Anyone buy or try this one? 👀,a close up of a jar of candles on a store shelf,77b37fb1987d8d0d8aff7cadd6942b95,/r/bathandbodyworks/comments/13n9x6y/anyone_bu...,https://i.redd.it/paiuac5fk31b1.jpg,13n9x6y.jpg,data/image/13n9x6y.jpg,CandleDiffusion,True,False,False,[]
39628,13nc2ad,bathandbodyworks,TFCosplayer92,Just a SMALL Firecracker Pop Haul 🤣,arafed items are laid out on the floor in a room,e4e276e4dbd240d546248f2951212e2e,/r/bathandbodyworks/comments/13nc2ad/just_a_sm...,https://i.redd.it/lf40r0xk041b1.jpg,13nc2ad.jpg,data/image/13nc2ad.jpg,CandleDiffusion,True,False,False,[]
39629,13nczq7,bathandbodyworks,blondenextdoor30,Happy Saturday from the West Coast!🌞🩶🍾 What’s ...,candles are lit on a counter top with a gold lid,8bcf2db56ca1390ad579fa44a0d5716d,/r/bathandbodyworks/comments/13nczq7/happy_sat...,https://i.redd.it/7b78c8l2841b1.jpg,13nczq7.jpg,data/image/13nczq7.jpg,CandleDiffusion,True,False,False,[]
39630,13nd8ob,bathandbodyworks,angry_owl253,Sunset glow!,there is a bottle of liquid and a bag of soap ...,20357839c2d8e688ffd7deba0bb300eb,/r/bathandbodyworks/comments/13nd8ob/sunset_glow/,https://i.redd.it/qr929v34a41b1.jpg,13nd8ob.jpg,data/image/13nd8ob.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 609 ms
Wall time: 1.32 s


In [21]:
%%time

temp_current = current
temp_current.set_index('id', inplace=True, drop=False)

temp_new = dropped
temp_new.set_index('id', inplace=True, drop=False)

temp_new.loc[~temp_new.index.isin(temp_current.index)]

CPU times: total: 15.6 ms
Wall time: 13 ms


,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
13nhxcp,13nhxcp,SFWRedheads,Middle-Ad4300,Hello 👋🏻,arafed woman sitting in a car with her legs cr...,955d683562bc7177e5369c129030c888,/r/SFWRedheads/comments/13nhxcp/hello/,https://i.redd.it/1g3nvmrwv31b1.jpg,13nhxcp.jpg,data/image/13nhxcp.jpg,RedHeadDiffusion,True,False,False,[]
13nifj6,13nifj6,SFWRedheads,arayofsexysunshine,What a crazy and touching time I’ve had on the...,arafed woman with red hair wearing a green jac...,f59f2c71f8c64da25432c72fa2ddd099,/r/SFWRedheads/comments/13nifj6/what_a_crazy_a...,https://i.redd.it/f2anwulzh51b1.jpg,13nifj6.jpg,data/image/13nifj6.jpg,RedHeadDiffusion,True,False,False,[]
13njz51,13njz51,SFWRedheads,madelynlouise,Went to the grocery store in this,there is a woman taking a selfie in a mirror,7e70167383380384e1f6fe47455faf0f,/r/SFWRedheads/comments/13njz51/went_to_the_gr...,https://i.redd.it/wcr5fnxcw51b1.jpg,13njz51.jpg,data/image/13njz51.jpg,RedHeadDiffusion,True,False,False,[]
13nka7r,13nka7r,SFWRedheads,celluxxx,Felt cute in this outfit ❤️,there is a woman sitting on a bed with a red d...,49bed56ba79c6c236ca818ed33b2cd04,/r/SFWRedheads/comments/13nka7r/felt_cute_in_t...,https://i.redd.it/m1lefhhaz51b1.jpg,13nka7r.jpg,data/image/13nka7r.jpg,RedHeadDiffusion,True,False,False,[]
13np8z0,13np8z0,SFWRedheads,Junior-Midnight-8241,Do I look innocent? 👀,arafed woman with long red hair and white top ...,17c5c634443b786c265dbe4bded9f20f,/r/SFWRedheads/comments/13np8z0/do_i_look_inno...,https://i.redd.it/nf7kb3tub71b1.jpg,13np8z0.jpg,data/image/13np8z0.jpg,RedHeadDiffusion,True,False,False,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


In [22]:
%%time

concat = pd.concat([current, dropped])

display("== Concatenated Data ==")
display(concat.shape)
display(concat)

'== Concatenated Data =='

(39719, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 109 ms
Wall time: 74 ms


In [23]:
%%time

file_list_ = file_system.ls("data/image")
concat['path'] = concat.progress_apply(lambda x: functions.fix_path(x, file_list), axis=1)

display("== Concatenated Data With Path ==")
display(concat.shape)
display(concat)

Progress: 100%|██████████| 39719/39719 [01:19<00:00, 501.46it/s] 


'== Concatenated Data With Path =='

(39719, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
id,,,,,,,,,,,,,,,
100rn7k,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1013bdt,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
105mekt,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
105qvgl,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
105rpcj,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
13no1on,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
13nhylw,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
13nlz14,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 2min 18s
Wall time: 2min 32s


In [24]:
%%time

back_up_name = f"data/parquet/back_{dt.datetime.timestamp(dt.datetime.now())}.parquet"
display(f"== Writing Back-Up {back_up_name} ==")
current.to_parquet(back_up_name, schema=schema, filesystem=file_system)

'== Writing Back-Up data/parquet/back_1684671254.881872.parquet =='

CPU times: total: 516 ms
Wall time: 3.11 s


In [25]:
%%time

concat.to_parquet("data/parquet/back.parquet", schema=schema, filesystem=file_system)
new = pd.read_parquet("data/parquet/back.parquet", engine='pyarrow', schema=schema, filesystem=file_system)

display("== New Data ==")
display(new.shape)
display(new)

'== New Data =='

(39719, 15)

,id,subreddit,author,title,caption,hash,permalink,original_url,image_name,path,model,exists,curated,accept,tags
0,100rn7k,AmIhotAF,veritynicole,"hey, hows your new year going (23F)",a woman in a white shirt and black pants is po...,4bd00c19fa0ff2ade855e6d364b0760b,/r/AmIhotAF/comments/100rn7k/hey_hows_your_new...,https://i.redd.it/n7r47s0gkh9a1.jpg,100rn7k.jpg,data/image/100rn7k.jpg,SexyDiffusion,True,True,False,[]
1,1013bdt,AmIhotAF,RaulDea9286,36F - ITALIAN,arafed image of a woman in a bikini top,7c0d158cba8654ef1c635cbc5471d597,/r/AmIhotAF/comments/1013bdt/36f_italian/,https://i.redd.it/bg0wwdlt5k9a1.jpg,1013bdt.jpg,data/image/1013bdt.jpg,SexyDiffusion,True,True,True,[]
2,105mekt,AmIhotAF,lindaniz,interesting in good forward relationship (f24),a close up of a woman with red hair and a whit...,ba4a0962cca2266a741e1e1700589c04,/r/AmIhotAF/comments/105mekt/interesting_in_go...,https://i.redd.it/4avjshsz8naa1.jpg,105mekt.jpg,data/image/105mekt.jpg,SexyDiffusion,True,True,True,[]
3,105qvgl,AmIhotAF,CaitVLove11,Laughing is my favorite 😆,a woman in a blue tank top and shorts is smili...,27bfe82c37314a0bcf02ab72eaf3a9e5,/r/AmIhotAF/comments/105qvgl/laughing_is_my_fa...,https://i.redd.it/2pulzr0lxmaa1.jpg,105qvgl.jpg,data/image/105qvgl.jpg,SexyDiffusion,True,True,True,[]
4,105rpcj,AmIhotAF,Flashy-Desk1858,[f22] What do you think when you see me?,a woman in a blue bikini top and a blue bra top,329eb42b8267fa1cc2980da8e48bcef1,/r/AmIhotAF/comments/105rpcj/f22_what_do_you_t...,https://i.redd.it/rz68pf934naa1.jpg,105rpcj.jpg,data/image/105rpcj.jpg,SexyDiffusion,True,True,True,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39714,13no1on,DLAH,Majestic_Painter8660,"Tight dress, legs and heels",there is a woman sitting on a chair in a room,dbab04d6a37e17e6cc6df9cfc16ed51e,/r/DLAH/comments/13no1on/tight_dress_legs_and_...,https://i.redd.it/0pq456mci51b1.jpg,13no1on.jpg,data/image/13no1on.jpg,PrettyGirlDiffusion,True,False,False,[]
39715,13nhylw,bathandbodyworks,BathbeautyXO,(Late) February/valentines empties! 💕❤️💖,there are many different types of skin care pr...,7d79ab2ad5d7fe73454db01b4351388c,/r/bathandbodyworks/comments/13nhylw/late_febr...,https://i.redd.it/t2yvxblvd51b1.jpg,13nhylw.jpg,data/image/13nhylw.jpg,CandleDiffusion,True,False,False,[]
39716,13nlz14,bathandbodyworks,Fresh-Blueberry-milk,figured this would be appreciated 😏 also what ...,someone is holding a bottle of lotion in their...,8b7dfc23aeb352024f0a696ab49351ae,/r/bathandbodyworks/comments/13nlz14/figured_t...,https://i.redd.it/or6fujdmf61b1.jpg,13nlz14.jpg,data/image/13nlz14.jpg,CandleDiffusion,True,False,False,[]
39717,13npfx3,bathandbodyworks,singshopsleep,All of Today's Offers,a poster with a variety of products for sale o...,635af0f6bb1a2b81c9ce46bd29374d72,/r/bathandbodyworks/comments/13npfx3/all_of_to...,https://i.redd.it/skqywk3qd71b1.jpg,13npfx3.jpg,data/image/13npfx3.jpg,CandleDiffusion,True,False,False,[]


CPU times: total: 1 s
Wall time: 3.72 s


In [26]:
!jupyter notebook stop

^C


Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-collection\venv\lib\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\Users\AJ Stangl\AppData\Local\Programs\Python\Python39\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
Error processing line 1 of D:\code\repos\simple-co